In [1]:
import re
import numpy as np
import math
import attr
import urllib.request
import itertools as it
import hashlib
import os

from collections import Counter, defaultdict, namedtuple, deque
from itertools   import permutations, combinations, chain, cycle, product, islice, takewhile
from heapq       import heappop, heappush

In order to have the Notebook automatically download

In [2]:
def Input(day, year='20XX'):
    """
    Open this day's input file.
    Optionally, specify the year.
    
    If it is not available locally, download and save the file.
    This uses the session variable which should be copied out of your browsers
    Cookie for adventofcode.com
    """
    filename = f'input/{year}/{day}.txt'
    try:
        return open(filename)
    except FileNotFoundError:
        return download_input(day, year)

def download_input(day, year):
    session = os.environ['AOCSESSION']
    filename = f'input/{year}/{day}.txt'
    try:
        print(f'Downloading input file for Advent of Code {year} - Day {day}')
        req = urllib.request.Request(
            f'http://adventofcode.com/{year}/day/{day}/input',
            headers={'Cookie':f'session={session}'})
        res = urllib.request.urlopen(req)
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        with open(f'input/{year}/{day}.txt', 'w') as w:
            result = res.read().decode('utf-8')
            w.write(result)
        return open(filename)
    except urllib.request.HTTPError:
        print('Downloading file failed.')
        return [f'Downloading input file for Advent of Code {year} - Day {day} failed.']

### Integer Functions

These were originally made for some Project Euler questions that I did in Clojure, adapted to Python.

They mostly revolve around splitting an integer into its individual digits which may come in handy, we shall see!

In [3]:
def len_int(num):
    "Return the number of digits in an int"
    return math.floor(math.log10(abs(num))) + 1

def first_int(num, preserve_sign=False):
    "Return the first digit of a number. Similar to % 10 to get the last digit."
    sign = 1
    if preserve_sign and num < 0:
        sign = -1
    return sign * (abs(num) // 10**(len_int(num)-1))

def split_int(num):
    "Split an int into an array of its digits"
    num = int(num)
    nums = []
    while num:
        digit = num % 10
        nums.append(digit)
        num //= 10
    nums.reverse()
    return nums

def reverse_int(num):
    "Reverse an ints digits"
    result = 0
    while num:
        digit = num % 10
        result = result * 10 + digit
        num //= 10
    return result

def join_int(nums):
    "Join an array of digits into an int"
    result = 0
    for num in nums:
        result *= 10
        result += num
    return result

In [4]:
assert len_int(1) == 1
assert len_int(283) == 3
assert reverse_int(123) == 321
assert split_int(123) == [1, 2, 3]
assert join_int([1, 2, 3]) == 123
assert first_int(48329) == 4
assert first_int(1) == 1
assert first_int(-83) == 8
assert first_int(-83, preserve_sign=True) == -8

### String Functions

`rotn` allows you to rotate a string n characters forward, allowing for techniques such as the traditional rot13 and more

In [5]:
def rotn(text, n, alphabet='abcdefghijklmnopqrstuvwxyz'):
    "Rotate the text n characters forward"
    n = n % len(alphabet)
    tr = str.maketrans(alphabet, alphabet[n:] + alphabet[:n])
    return text.translate(tr)

def md5(text):
    "Hashes the input text using the md5 algorithm."
    return hashlib.md5(bytes(str(text), 'utf-8')).hexdigest()

def subsequences(seq, n):
    return [seq[i:i+n] for i in range(0, len(seq) + 1 - n)]

In [6]:
assert rotn('abc', 2) == 'cde'
assert subsequences('test', 2) == ['te', 'es', 'st']
assert len(md5('hi')) == len(md5('otherword'))

In [7]:
# Input Parsing    
def parse_ints(text): 
    "All the integers anywhere in text."
    return [int(x) for x in re.findall(r'-?\d+', text)]

def read_letters(file):
    "Get a list of all of the letters in the input file."
    return list(file.read().strip())

def read_words(file):
    "Get a list of all of the letters in the input file."
    return [line.strip().split() for line in file]

def read_numbers(file):
    "If the file is just 123456 return [1, 2, 3, 4, 5, 6]"
    return [int(n) for n in file.read().strip()]

In [8]:
assert parse_ints('it takes 12 into -42, then 39.4') == [12, -42, 39, 4]

In [9]:
def peek(iterable, amount=10):
    "Returns the first 10 elements of an iterable, usually to preview the input."
    return list(iterable)[:amount]

def first(iterable, default=None):
    "Return the first element of an iterable, or default if it is empty."
    return next(iter(iterable), default)

def first_true(iterable, pred=None, default=None):
    """Returns the first true value in the iterable.
    If no true value is found, returns *default*
    If *pred* is not None, returns the first item
    for which pred(item) is true."""
    # first_true([a,b,c], default=x) --> a or b or c or x
    # first_true([a,b], fn, x) --> a if fn(a) else b if fn(b) else x
    return next(filter(pred, iterable), default)

def upto(iterable, maxval):
    "From a monotonically increasing iterable, generate all the values <= maxval."
    # Why <= maxval rather than < maxval? In part because that's how Ruby's upto does it.
    return takewhile(lambda x: x <= maxval, iterable)

def nth(iterable, n, default=None):
    "Returns the nth item of iterable, or a default value"
    return next(islice(iterable, n, None), default)

def transpose(matrix): return tuple(zip(*matrix))

cat = ''.join

def join(iterable, sep=''):
    "Join the itemsin iterable, converting each to a string first."
    return sep.join(map(str, iterable))

inf = float('inf')

def grep(pattern, lines):
    "Yield lines that match given pattern."
    for line in lines:
        if re.search(pattern, line):
            yield line

def groupby(iterable, key=lambda it: it):
    "Return a dic whose keys are key(it) and whose values are all the elements of iterable with that key."
    dic = defaultdict(list)
    for it in iterable:
        dic[key(it)].append(it)
    return dic

def quantify(iterable, pred=bool):
    "Count how many times the predicate is true."
    return sum(map(pred, iterable))

def powerset(iterable):
    "Yield all subsets of items."
    items = list(iterable)
    for r in range(len(items)+1):
        for c in combinations(items, r):
            yield c
            
def trace1(f):
    "Print a trace of the input and output of a function on one line."
    def traced_f(*args):
        result = f(*args)
        print('{}({}) = {}'.format(f.__name__, ', '.join(map(str, args)), result))
        return result
    return traced_f

In [10]:
assert first([0,1,2,3,4,5]) == 0
assert nth([0,1,2,3,4,5], 3) == 3
assert cat(['hi', 'dude']) == 'hidude'
assert list(grep('test', ['excellent lets', 'test this'])) == ['test this']

In [11]:
@attr.s(hash=True)
class Point():
    x: int = attr.ib(default=0)
    y: int = attr.ib(default=0)
    
    def __add__(self, other):
        return Point(self.x + other.x, self.y + other.y)

In [12]:
Bounds = namedtuple('Bounds', 'left top right bottom')

def point_within_bounds(bounds, point):
    "Test if the point is within the given bounds."
    x, y = attr.astuple(point)
    return not (x > bounds.right  or x < bounds.left
                or y < bounds.top or y > bounds.bottom)

def neighbors4(point, bounds=None): 
    """
    The four neighbors (without diagonals).
    Bounds are in the format of Bounds(left, top, right, bottom) or None
    """
    x, y = attr.astuple(point)
    points = [Point(x+1, y), Point(x-1, y),
              Point(x, y+1), Point(x, y-1)]
    if bounds:
        return tuple(filter(lambda p: point_within_bounds(bounds, p), points))
    else:
        return tuple(points)

def neighbors8(point, bounds=None): 
    """
    The eight neighbors (with diagonals).
    Bounds are in the format of Bounds(left, top, right, bottom) or None
    """
    x, y = attr.astuple(point)
    points = [Point(x+1, y),   Point(x-1, y),
            Point(x, y+1),   Point(x, y-1),
            Point(x+1, y+1), Point(x-1, y-1),
            Point(x+1, y-1), Point(x-1, y+1)]
    if bounds:
        return tuple(filter(lambda p: point_within_bounds(bounds, p), points))
    else:
        return tuple(points)

def cityblock_distance(p, q=Point(0, 0)): 
    "City block distance between two points."
    return abs(p.x - q.x) + abs(p.y - q.y)

def euclidean_distance(p, q=Point(0, 0)): 
    "Euclidean (hypotenuse) distance between two points."
    return math.hypot(p.x - q.x, p.y - q.y)

In [13]:
assert len(neighbors4(Point(0, 3))) == 4
assert len(neighbors4(Point(0, 0), Bounds(0, 0, 1, 1))) == 2

In [14]:
assert len(neighbors8(Point(0, 0), Bounds(0, 0, 1, 1))) == 3